# Lexicographic (Hierarchical) Optimization

this is for minimal spending as thats what most people building a pc has in mind anyways

In [1]:
import pandas as pd
from typing import List, Dict, Union

In [2]:
path = "../data/parts/"

In [32]:
# Load datasets
games_df = pd.read_csv("../data/games/top300.csv")

cpu_df = pd.read_csv(path + "CPU_Data.csv")
gpu_df = pd.read_csv(path + "GPU_Data.csv")
ram_df = pd.read_csv(path + "RAM_Data.csv")

mobo_df = pd.read_csv(path + "MOBO_Data.csv")
psu_df = pd.read_csv(path + "PSU_Data.csv")
case_df = pd.read_csv(path + "Case_Data.csv")

# storage_df = pd.read_csv(data_path + "Storage_Data.csv")

def safe_to_numeric(value):
    try:
        return pd.to_numeric(value)
    except (ValueError, TypeError):
        return value  

# Convert columns to numeric where possible
games_df = games_df.apply(safe_to_numeric)
cpu_df = cpu_df.apply(safe_to_numeric)
gpu_df = gpu_df.apply(safe_to_numeric)
ram_df = ram_df.apply(safe_to_numeric)
case_df = case_df.apply(safe_to_numeric)
mobo_df = mobo_df.apply(safe_to_numeric)
psu_df = psu_df.apply(safe_to_numeric)

# storage_df = storage_df.apply(safe_to_numeric)

In [ ]:
# Lexicographic (Hierarchical) Optimization 
# (before renaming some variables and moving around some things)
# This was the first itteration of the algo, it is the slowest and is just pure brute force
# I KNOW this will take a long time to work so I didn't even bother trying to record its calculation time

def select_pc_build(budget: float, selected_game_names: List[str]) -> Union[str, Dict]:
    selected_games = games_data[games_data['name'].isin(selected_game_names)]
    if selected_games.empty:
        return "❌ No matching games found in the database."

    # Step 1: Required specs (max among selected games)
    required_cpu_score = selected_games['CPU'].max()
    required_gpu_score = selected_games['GPU'].max()
    required_ram = max(selected_games['memory'].max(), 8)  # Enforce at least 8GB RAM

    min_cpu_score = 0.8 * required_cpu_score
    min_gpu_score = 0.8 * required_gpu_score

    valid_builds = []

    # Step 2: Iterate and evaluate all valid builds
    for _, cpu in cpu_data.iterrows():
        for _, mobo in mobo_data.iterrows():
            if cpu['Chipset'] != mobo['Socket']:
                continue

            for _, gpu in gpu_data.iterrows():
                if gpu['Score'] < min_gpu_score:
                    continue

                for _, psu in psu_data.iterrows():
                    if psu['Wattage'] < gpu['Recommended Power']:
                        continue

                    for _, case in case_data.iterrows():
                        if mobo['Size'] not in case['Size']:
                            continue

                        for _, ram in ram_data.iterrows():
                            for sticks in [1, 2, 4]:
                                total_ram = ram['Capacity (GB)'] * sticks
                                if total_ram < required_ram:
                                    continue
                                if sticks > mobo['RAM Slot']:
                                    continue
                                if ram['DDR'] != mobo['DDR']:
                                    continue

                                total_price = (
                                    cpu['Price'] + mobo['Price'] + gpu['Price'] +
                                    psu['Price'] + case['Price'] + ram['Price'] * sticks
                                )

                                if total_price > budget:
                                    continue

                                valid_builds.append({
                                    "CPU": cpu['Name'],
                                    "MOBO": mobo['Name'],
                                    "GPU": gpu['Name'],
                                    "PSU": psu['Name'],
                                    "Case": case['Name'],
                                    "RAM": f"{sticks}x {ram['Name']} ({total_ram}GB)",
                                    "Total_Price": total_price,
                                    "CPU_Score": cpu['Score'],
                                    "GPU_Score": gpu['Score']
                                })

    # Step 3: Evaluate Results
    if not valid_builds:
        if cpu_data['Score'].max() < min_cpu_score or gpu_data['Score'].max() < min_gpu_score:
            return "❌ Budget is too low to meet the minimum CPU or GPU performance (-20% margin)."
        else:
            return "⚠️ Build may be feasible in terms of performance, but no compatible parts fit within the budget."

    # Step 4: Lexicographic sorting (GPU > CPU > Cost)
    sorted_builds = sorted(valid_builds, key=lambda x: (-x['GPU_Score'], -x['CPU_Score'], x['Total_Price']))
    best = sorted_builds[0]

    warnings = []
    if best['CPU_Score'] < required_cpu_score or best['GPU_Score'] < required_gpu_score:
        warnings.append("⚠️ Build is within 20% performance margin but may stutter in some games.")

    return {
        "Build": best,
        "Warnings": warnings
    }

# Selecting a PC build
result = select_pc_build(budget=2500, selected_game_names=["PUBG: BATTLEGROUNDS", "Counter-Strike 2"])

# Display the result
if isinstance(result, str):
    print(result)
else:
    print("✅ Recommended Build:")
    for k, v in result["Build"].items():
        print(f"{k}: {v}")
    if result["Warnings"]:
        print("\nWarnings:")
        for w in result["Warnings"]:
            print(w)

In [ ]:
# Lexicographic (Hierarchical) Optimization
# This is after merging cpu and motherboard to decrease calculation time
# It didnt finish calculating after 10 minutes so just stopped it from there
# Its way worse than ILP to even consider using at this level
# Ill try to preprocess pc part compatibility first 

def select_pc_build(budget: float, selected_games: List[str]) -> Union[str, Dict]:
    filtered_games = games_df[games_df['name'].isin(selected_games)]
    if filtered_games.empty:
        return "❌ No matching games found in the database."

    # Required specs from selected games
    required_cpu_score = filtered_games['CPU'].max()
    required_gpu_score = filtered_games['GPU'].max()
    required_ram_gb = max(filtered_games['memory'].max(), 8)

    min_cpu_score = 0.8 * required_cpu_score
    min_gpu_score = 0.8 * required_gpu_score

    # Pre-filter GPUs and PSUs to reduce search space
    gpu_filtered = gpu_df[gpu_df['Score'] >= min_gpu_score]
    psu_filtered = psu_df.copy()

    # Compatible CPU-MOBO pairs (socket/chipset match)
    cpu_mobo = pd.merge(
        cpu_df, mobo_df,
        left_on='Socket', right_on='Socket',
        suffixes=('_cpu', '_mobo')
    )

    # Filter RAM sticks that fit motherboard DDR
    ram_filtered = ram_df.copy()

    # Prepare list for valid builds
    valid_builds = []

    # Iterate over CPU-MOBO compatible pairs
    for _, cpu_mobo_row in cpu_mobo.iterrows():
        cpu_score = cpu_mobo_row['Score']
        mobo_ram_slots = cpu_mobo_row['RAM Slot']
        mobo_ddr = cpu_mobo_row['DDR']
        mobo_size = cpu_mobo_row['Size']

        # Skip CPU if below minimum CPU score needed (optional pruning)
        if cpu_score < min_cpu_score:
            continue

        # Filter RAM compatible with this motherboard DDR
        ram_compatible = ram_filtered[ram_filtered['DDR'] == mobo_ddr]

        # Filter cases compatible with motherboard size
        case_compatible = case_df[case_df['Size'].str.contains(mobo_size)]

        # Iterate GPUs that pass min GPU score
        for _, gpu_row in gpu_filtered.iterrows():
            gpu_score = gpu_row['Score']
            gpu_power_req = gpu_row['Recommended Power']

            # Filter PSUs that can support this GPU power requirement
            psu_compatible = psu_filtered[psu_filtered['Wattage'] >= gpu_power_req]

            # Iterate PSUs
            for _, psu_row in psu_compatible.iterrows():

                # Iterate compatible cases
                for _, case_row in case_compatible.iterrows():

                    # Iterate compatible RAM and possible stick counts
                    for _, ram_row in ram_compatible.iterrows():
                        for sticks in [1, 2, 4]:
                            total_ram = ram_row['Capacity (GB)'] * sticks

                            if total_ram < required_ram_gb:
                                continue
                            if sticks > mobo_ram_slots:
                                continue

                            # Compute total price
                            total_price = (
                                cpu_mobo_row['Price_cpu'] + cpu_mobo_row['Price_mobo'] +
                                gpu_row['Price'] + psu_row['Price'] + case_row['Price'] +
                                ram_row['Price'] * sticks
                            )

                            if total_price > budget:
                                continue

                            valid_builds.append({
                                "CPU": cpu_mobo_row['Name_cpu'],
                                "MOBO": cpu_mobo_row['Name_mobo'],
                                "GPU": gpu_row['Name'],
                                "PSU": psu_row['Name'],
                                "Case": case_row['Model'],
                                "RAM": f"{sticks}x {ram_row['Name']} ({total_ram}GB)",
                                "Total_Price": total_price,
                                "CPU_Score": cpu_score,
                                "GPU_Score": gpu_score
                            })

    if not valid_builds:
        if cpu_df['Score'].max() < min_cpu_score or gpu_df['Score'].max() < min_gpu_score:
            return "❌ Budget is too low to meet the minimum CPU or GPU performance (-20% margin)."
        else:
            return "⚠️ Build may be feasible in terms of performance, but no compatible parts fit within the budget."

    # Lexicographic sorting (GPU > CPU > Price)
    sorted_builds = sorted(valid_builds, key=lambda b: (-b['GPU_Score'], -b['CPU_Score'], b['Total_Price']))
    best_build = sorted_builds[0]

    warnings = []
    if best_build['CPU_Score'] < required_cpu_score or best_build['GPU_Score'] < required_gpu_score:
        warnings.append("⚠️ Build is within 20% performance margin but may stutter in some games.")

    return {
        "Build": best_build,
        "Warnings": warnings
    }

# Selecting a PC build
result = select_pc_build(budget=2500, selected_games=["PUBG: BATTLEGROUNDS", "Counter-Strike 2"])

# Display the result
if isinstance(result, str):
    print(result)
else:
    print("✅ Recommended Build:")
    for part, description in result["Build"].items():
        print(f"{part}: {description}")
    if result["Warnings"]:
        print("\nWarnings:")
        for w in result["Warnings"]:
            print(w)

In [ ]:
# Before merging the parts I considered doing a Cartesian product but I realized its not scaleable at all
# and that the number of parts we already have is too big for it (we don't even have that many)

In [28]:
# Pre processing the compability of parts
# CPU-MOBO compatibility -  by chipset/socket.
# MOBO-RAM compatibility -  by DDR version & RAM slots.
# MOBO-Case compatibility - by size.
# GPU-PSU compatibility -   by required power.

# Preprocess CPU–MOBO compatibility
cpu_mobo_df = pd.merge(
    cpu_df, mobo_df,
    left_on='Socket', right_on='Socket',
    suffixes=('_cpu', '_mobo')
)

# Preprocess MOBO–RAM compatibility
mobo_ram_df = pd.merge(
    mobo_df, ram_df,
    left_on='DDR', right_on='DDR',
    suffixes=('_mobo', '_ram')
)

# Preprocess MOBO–Case compatibility
mobo_case_pairs = []
for _, mobo in mobo_df.iterrows():
    for _, case in case_df.iterrows():
        if mobo['Size'] in case['Size']:
            mobo_case_pairs.append((mobo['Name'], case['Model']))

# Preprocess GPU–PSU compatibility
gpu_psu_pairs = []
for _, gpu in gpu_df.iterrows():
    for _, psu in psu_df.iterrows():
        if psu['Wattage'] >= gpu['Recommended Power']:
            gpu_psu_pairs.append((gpu['Name'], psu['Name']))


In [ ]:
# Refactored optimization after preprocessing the parts
# Even after another 10 minutes its still not done processing
# This makes the Lexicographic Optimization a bad choice because
# 1: Excessive Computation Time - It takes too long to process
# 2: Lack of Scalability - It isnt scalable because it requires you to have a small dataset due to its brute force nature

def select_pc_build(budget: float, selected_games: List[str]) -> Union[str, Dict]:
    filtered_games = games_df[games_df['name'].isin(selected_games)]
    if filtered_games.empty:
        return "❌ No matching games found in the database."

    required_cpu_score = filtered_games['CPU'].max()
    required_gpu_score = filtered_games['GPU'].max()
    required_ram = max(filtered_games['memory'].max(), 8)

    min_cpu_score = 0.8 * required_cpu_score
    min_gpu_score = 0.8 * required_gpu_score

    valid_builds = []

    # Iterate over compatible CPU-MOBO pairs
    for _, row in cpu_mobo_df.iterrows():
        cpu = row
        mobo = row  # because cpu_mobo_df contains both

        if cpu['Score'] < min_cpu_score:
            continue

        # Get compatible RAMs from mobo_ram_df
        compatible_rams = mobo_ram_df[mobo_ram_df['Name_mobo'] == mobo['Name_mobo']]
        for _, ram_row in compatible_rams.iterrows():
            ram = ram_row
            for sticks in [1, 2, 4]:
                total_ram = ram['Capacity (GB)'] * sticks
                if total_ram < required_ram:
                    continue
                if sticks > mobo['RAM Slot']:
                    continue

                ram_total_price = ram['Price_ram'] * sticks
                ram_description = f"{sticks}x {ram['Name_ram']} ({total_ram}GB)"

                # Find cases compatible with this MOBO
                case_names = [case for moboname, case in mobo_case_pairs if moboname == mobo['Name_mobo']]
                filtered_cases = case_df[case_df['Model'].isin(case_names)]

                for _, case in filtered_cases.iterrows():
                    # Iterate GPU–PSU compatible pairs
                    for gpu_name, psu_name in gpu_psu_pairs:
                        gpu = gpu_df[gpu_df['Name'] == gpu_name].iloc[0]
                        psu = psu_df[psu_df['Name'] == psu_name].iloc[0]

                        if gpu['Score'] < min_gpu_score:
                            continue

                        total_price = (
                            cpu['Price_cpu'] + mobo['Price_mobo'] + gpu['Price'] +
                            psu['Price'] + case['Price'] + ram_total_price
                        )

                        if total_price > budget:
                            continue

                        valid_builds.append({
                            "CPU": cpu['Name_cpu'],
                            "MOBO": mobo['Name_mobo'],
                            "GPU": gpu['Name'],
                            "PSU": psu['Name'],
                            "Case": case['Model'],
                            "RAM": ram_description,
                            "Total_Price": total_price,
                            "CPU_Score": cpu['Score'],
                            "GPU_Score": gpu['Score']
                        })

    # Evaluate builds
    if not valid_builds:
        if cpu_df['Score'].max() < min_cpu_score or gpu_df['Score'].max() < min_gpu_score:
            return "❌ Budget is too low to meet the minimum CPU or GPU performance (-20% margin)."
        else:
            return "⚠️ Build may be feasible in terms of performance, but no compatible parts fit within the budget."

    # Sort lexicographically (GPU > CPU > Price)
    sorted_builds = sorted(valid_builds, key=lambda x: (-x['GPU_Score'], -x['CPU_Score'], x['Total_Price']))
    best = sorted_builds[0]

    warnings = []
    if best['CPU_Score'] < required_cpu_score or best['GPU_Score'] < required_gpu_score:
        warnings.append("⚠️ Build is within 20% performance margin but may stutter in some games.")

    return {
        "Build": best,
        "Warnings": warnings
    }

# Selecting a PC build
result = select_pc_build(budget=2500, selected_games=["PUBG: BATTLEGROUNDS", "Counter-Strike 2"])

# Display the result
if isinstance(result, str):
    print(result)
else:
    print("✅ Recommended Build:")
    for part, description in result["Build"].items():
        print(f"{part}: {description}")
    if result["Warnings"]:
        print("\nWarnings:")
        for w in result["Warnings"]:
            print(w)